<a href="https://colab.research.google.com/github/EmaJuarezDev/CD-1C2025/blob/main/EXCEL_REGRESION_LOGISTICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examen Práctico

#### 01-3900 | Ciencia de datos | 2025

Alumno: Pablo M. Ferreira

## Enunciado

Se tienen un dataset con datos de pacientes internados en un hospital (TP_Virus_Alumnos.csv). La clase de interes (1) refiere a la presencia de un virus. El virus tiene normalmente una gravedad leve/baja y el tratamiento suele ser invasivo. Datos como nombre y apellido han sido eliminados y los valores tanto en sangre (BLD), hormonales u otros análisis sobre reactivos han sido alterados en sus valores para preservar la privacidad. Se aclara que no se ha modificado su capacidad predictiva (Si es que la tienen).


Para su conocimiento: </BR>
Datos generales de Edad, Peso y condición laboral (Activo, Pasivo etc).
Datos medidos en hospital:</BR>
BLD: Sangre</BR>
LVL: Hormonales</BR>
REC: Otros análisis</BR>

Se pide obtener con los datos disponibles el mejor modelo posible que prediga la presencia o ausencia del virus.
Dado que el tratamiento es invasivo y la gravedad es moderada se requiere "atrapar" tantos "1" como sea posible y minimizar los falsos positivos para evitar que reciban un tratamiento de estas caracteristicas personas que no presentan el virus. Intente obtener el mejor modelo que maximice la métrica que considere correspondiente.



## Como desarrollar el exámen

A partir del dataset realice todas las acciones para poder llegar al mejor modelo, explique brevemente en los fundamentos de sus transformaciones o acciones en general.

La nota derivará de: </BR>
1.La calidad de la clasificación realizada</BR>
2.La fundamentación de los pasos realizados</BR>
3.Lo sencillo de llevar a producción el desarrollo</BR>



Los docentes evaluaran su clasificador utilizando un conjunto de datos del dataset "fuera de la caja" (out of the box, al que usted no tiene acceso). Para minimizar la posible diferencia entre su medición y la medición del docente recuerde y aplique conceptos de test, validación cruzada y evite los errores comunes de sesgo de selección y fuga de datos (Sklearn "10. Common pitfalls and recommended practices" disponible en "https://scikit-learn.org/stable/common_pitfalls.html)"   

Al final del notebook encontrará un bloque de código que lee la muestra adicional (a la que usted no tiene acceso) si PRODUCCION==True, en caso contrario solo lee una submuestra del conjunto original para validar que el código funciona. Desarrolle el notebook como considere para finalmente asignar el mejor clasificador o pipeline que usted haya obtenido remplazando en f_clf = None, None por su clasificador o pipeline. Si no utiliza un pipeline, implemente todas las transformaciones entre esa línea y la predicción final.

Persista modelos si realiza procesos que demoren (Mas de 10 minutos es mucho), alternativamente si quiere realizar búsquedas exhaustivas de hiperparametros o variables explicite el procedimiento y luego utilice los valores obtenidos para ajustar un clasificador/regresor y que los tiempos sean posibles en la corrección. Todas las herramientas vistas en clase están disponibles. Verifique que los docentes pueden ejecutar su clasificador / regresor usando el código adjunto y los datos "fuera de la caja" para validar la calidad su modelo.  

En materiales del MIEL/GIT se adjuntan un notebooks con algunas ideas para automatizar el proceso (Pipelines/Transformadores customizados).

## Desarrollo

In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/Franxx20/ciencia-de-datos-tp-parcial/refs/heads/main/TP_Virus_Alumnos.csv", sep=",")
df = df.drop_duplicates() # Innecesario en este caso: el dataset no tiene duplicados

In [ ]:
print(df.isnull().sum())  # Tenemos nulos?

Edad        822
Genero     1682
Peso          0
Laboral       0
hijos         0
BLD01         0
REC1          0
REC2          0
REC3          0
REC4          0
REC5          0
BLD02         0
BLD03         0
LVL         527
target        0
dtype: int64


In [ ]:
print((df.isnull().sum() / len(df)*100).sort_values(ascending=False)) # Porcentaje de valores nulos por atributo

Genero     40.047619
Edad       19.571429
LVL        12.547619
Laboral     0.000000
hijos       0.000000
BLD01       0.000000
Peso        0.000000
REC1        0.000000
REC2        0.000000
REC4        0.000000
REC3        0.000000
REC5        0.000000
BLD02       0.000000
BLD03       0.000000
target      0.000000
dtype: float64


In [ ]:
# Utilizando matrices de correlación, pair-plots, etc. notamos que 'Edad' se puede imputar a partir de 'Peso' e 'hijos' (i.e.: las dos 'features' más cercanas).
# TODO: Qué tal si imputo con KNN con 2 neighbours?

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(n_nearest_features=2)
df['Edad'] = imputer.fit_transform(df[['Edad']])

# Por otro lado, la columna 'Genero' tiene cerca de un 40% de nulos, no sería honesto utilizarla
df = df.drop(columns=['Genero'])

# Finalmente, imputo LVL con base en otros atributos, al igual que para la edad
df['LVL'] = imputer.fit_transform(df[['LVL']])

print(df.isnull().sum())

Edad       0
Peso       0
Laboral    0
hijos      0
BLD01      0
REC1       0
REC2       0
REC3       0
REC4       0
REC5       0
BLD02      0
BLD03      0
LVL        0
target     0
dtype: int64


In [ ]:
print(df.dtypes) # Tipos de cada atributo

Edad       float64
Peso       float64
Laboral     object
hijos        int64
BLD01      float64
REC1       float64
REC2       float64
REC3       float64
REC4       float64
REC5       float64
BLD02      float64
BLD03      float64
LVL        float64
target       int64
dtype: object


In [ ]:
# Uso one hot encoding para el atributo 'laboral'

from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse_output=False, dtype=int)
encoded_features = onehot_encoder.fit_transform(df[['Laboral']])
feature_names = onehot_encoder.get_feature_names_out(['Laboral'])
encoded_df = pd.DataFrame(encoded_features, columns=feature_names, index=df.index)
df = pd.concat([df, encoded_df], axis=1)
df = df.drop(['Laboral'], axis=1)

print(df.head())

        Edad        Peso  hijos     BLD01      REC1       REC2       REC3  \
0   9.000000   36.410602      0  3.139714  6.929112  63.712640  73.880613   
1  58.000000  115.507184      4  2.953914  4.660136  64.582610  73.669319   
2  74.000000   81.249035      3  1.365200  3.005861  63.613454  75.103984   
3  38.658082   77.985589      0  2.329285  4.344642  64.433115  73.402436   
4   1.000000   13.446208      0 -0.062975  3.938741  65.122602  75.007982   

        REC4       REC5     BLD02     BLD03             LVL  target  \
0  15.087239  90.400193  2.015331  1.633836       51.411632       0   
1  14.503106  90.817947  0.973317 -0.439358  1000000.000000       0   
2  14.077988  91.741638  3.243344  0.380413  1000000.000000       1   
3  15.583846  91.741709  3.188290 -0.226046   227134.359847       0   
4  16.961593  90.668625  2.729202  0.589698       21.627303       1   

   Laboral_Activo  Laboral_Inactivo  Laboral_No_declara  
0               0                 1                 

In [ ]:
# Notamos dos grandes grupos de LVL, los menores a 0.2 millones y los mayores a 0.7 millones, aplicamos binding separando en 0.5 (valor arbitrario).

threshold = 0.5 * 1000000

def categorize_lvl(lvl):
  if lvl <= threshold:
    return 'low'
  elif lvl >= threshold:
    return 'high'

df['LVL_binding'] = df['LVL'].apply(categorize_lvl)
df = df.drop('LVL', axis=1)

# Ahora aplicamos one hot encoding al binding

onehot_encoder_binding = OneHotEncoder(sparse_output=False)
encoded_binding = onehot_encoder_binding.fit_transform(df[['LVL_binding']])
binding_feature_names = onehot_encoder_binding.get_feature_names_out(['LVL_binding'])
encoded_binding_df = pd.DataFrame(encoded_binding, columns=binding_feature_names, index=df.index)
df = pd.concat([df, encoded_binding_df], axis=1)
df = df.drop('LVL_binding', axis=1)

print(df.head())

        Edad        Peso  hijos     BLD01      REC1       REC2       REC3  \
0   9.000000   36.410602      0  3.139714  6.929112  63.712640  73.880613   
1  58.000000  115.507184      4  2.953914  4.660136  64.582610  73.669319   
2  74.000000   81.249035      3  1.365200  3.005861  63.613454  75.103984   
3  38.658082   77.985589      0  2.329285  4.344642  64.433115  73.402436   
4   1.000000   13.446208      0 -0.062975  3.938741  65.122602  75.007982   

        REC4       REC5     BLD02     BLD03  target  Laboral_Activo  \
0  15.087239  90.400193  2.015331  1.633836       0               0   
1  14.503106  90.817947  0.973317 -0.439358       0               0   
2  14.077988  91.741638  3.243344  0.380413       1               1   
3  15.583846  91.741709  3.188290 -0.226046       0               0   
4  16.961593  90.668625  2.729202  0.589698       1               0   

   Laboral_Inactivo  Laboral_No_declara  LVL_binding_high  LVL_binding_low  
0                 1              

In [ ]:
from sklearn.preprocessing import StandardScaler

float_cols = df.select_dtypes('float64').columns
scaler = StandardScaler()
df[float_cols] = scaler.fit_transform(df[float_cols])

print(df.head())


       Edad      Peso  hijos     BLD01      REC1      REC2      REC3  \
0 -1.320429 -1.457972      0  1.419903  2.916859 -0.309346  0.838158   
1  0.861135  1.608986      4  1.259974  0.644639  0.559778  0.633716   
2  1.573483  0.280631      3 -0.107525 -1.012002 -0.408436  2.021856   
3  0.000000  0.154092      0  0.722319  0.328693  0.410428  0.375487   
4 -1.676602 -2.348413      0 -1.336839 -0.077788  1.099246  1.928968   

       REC4      REC5     BLD02     BLD03  target  Laboral_Activo  \
0  0.068414  0.391485  0.664739  1.057077       0               0   
1 -0.494157  0.802441 -0.363867 -1.024958       0               0   
2 -0.903582  1.711104  1.876951 -0.201691       1               1   
3  0.546690  1.711173  1.822605 -0.810737       0               0   
4  1.873579  0.655549  1.369425  0.008487       1               0   

   Laboral_Inactivo  Laboral_No_declara  LVL_binding_high  LVL_binding_low  
0                 1                   0         -0.497767         0.497767 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = df.drop("target", axis=1)
Y = df["target"]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify = Y, random_state=42)

model = LogisticRegression(
    penalty='l1',
    C=0.01,
    solver='saga'
)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.89      0.80       748
           1       0.76      0.52      0.61       512

    accuracy                           0.74      1260
   macro avg       0.74      0.70      0.71      1260
weighted avg       0.74      0.74      0.72      1260



In [ ]:
# Intento mejorar los hiperparametros automáticamente

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score

# Define hyperparameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': [None, 'l1', 'l2', 'elasticnet'],
    'solver': ['saga']
}

# Grid search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='precision')
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and

{'C': 0.01, 'penalty': 'l1', 'solver': 'saga'}


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
25 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py", line 1203, in fit
    raise ValueError("l1_

## Evaluacion final - Docente + Alumno

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

PRODUCCION = False
best_clf = None #Asignar aqui el mejor clasificador/pipeline posible (previamente entrenado si es necesario)

#Leemos el dataset de evaluación, simulando producción
if PRODUCCION==False:
    df = pd.read_csv("TP_Virus_Alumnos.csv")
    _, df = train_test_split(df, test_size=0.3, random_state=42)
else:
    df = pd.read_csv("TP_Virus_Evaluacion.csv")
#Dividimos en target y predictoras

X_prod = df.drop("target", axis=1)
y_prod = df["target"]

#Transformaciones en caso de no ser un pipeline completo. Preferiblemente, el mejor pipeline debe incluir todas las transformaciones necesarias.
#Si es pipeline ejecutar el fit (tener en cuenta el tiempo). Si mediante exploración / busqueda exhaustiva se encontró el mejor clasificador
#harcodear sus hiperparametros (Adjuntar código y explicación de como se encontró el mejor clasificador).

#Evaluación final
y_pred = best_clf.predict(X_prod)
print(classification_report(y_prod, y_pred))